# Valetudo - Projeto Jupiter - 2019
Valetudo launch from Projeto Jupiter (University of São Paulo, Brazil).
Permission to use flight data given by Guilherme Fernandes, 2020


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Importing libraries
import matplotlib.pyplot as plt

from rocketpy import Environment, Flight, Rocket, SolidMotor

In [ ]:
plt.style.use("seaborn-v0_8-dark-palette")

## RocketPy Simulation


Define a dictionary with the inputs for the simulation


In [ ]:
parameters = {
    # Mass Details
    "rocket_mass": (8.257, 0.001),
    # Propulsion Details
    "impulse": (1415.15, 35.3),
    "burn_time": (5.274, 1),
    "nozzle_radius": (21.642 / 1000, 0.5 / 1000),
    "throat_radius": (8 / 1000, 0.5 / 1000),
    "grain_separation": (6 / 1000, 1 / 1000),
    "grain_density": (1707, 50),
    "grain_outer_radius": (21.4 / 1000, 0.375 / 1000),
    "grain_initial_inner_radius": (9.65 / 1000, 0.375 / 1000),
    "grain_initial_height": (120 / 1000, 1 / 1000),
    # Aerodynamic Details
    "inertia_I": (3.675, 0.03675),
    "inertia_Z": (0.007, 0.00007),
    "radius": (40.45 / 1000, 0.001),
    "distance_rocket_nozzle": (-1.024, 0.001),
    "distance_rocket_propellant": (-0.571, 0.001),
    "power_off_drag": (0.9081 / 1.05, 0.033),
    "power_on_drag": (0.9081 / 1.05, 0.033),
    "nose_length": (0.274, 0.001),
    "nose_distance_to_cm": (1.134, 0.001),
    "fin_span": (0.077, 0.0005),
    "fin_root_chord": (0.058, 0.0005),
    "fin_tip_chord": (0.018, 0.0005),
    "fin_distance_to_cm": (-0.906, 0.001),
    # Launch and Environment Details
    "wind_direction": (0, 2),
    "wind_speed": (1, 0.033),
    "inclination": (84.7, 1),
    "heading": (53, 2),
    "rail_length": (5.7, 0.0005),
    # Parachute Details
    "cd_s_drogue": (0.349 * 1.3, 0.07),
    "lag_rec": (1, 0.5),
    # Electronic Systems Details
    "lag_se": (0.73, 0.16),
}

### Environment


Define the `Environment` object


In [ ]:
# Environment conditions
env = Environment(
    date=(2019, 8, 10, 21),
    latitude=-23.363611,
    longitude=-48.011389,
    elevation=668,
)

env.set_atmospheric_model(
    type="Reanalysis",
    file="../../tests/fixtures/acceptance/PJ_Valetudo/valetudo_weather_data_ERA5.nc",
    dictionary="ECMWF",
)


Visualize the `Environment` object


In [ ]:
env.info()

### Motor


Define the `SolidMotor` object


In [ ]:
keron = SolidMotor(
    thrust_source="../../data/motors/projeto-jupiter/keron_thrust_curve.csv",
    burn_time=parameters.get("burn_time")[0],
    dry_mass=0.001,
    dry_inertia=(0, 0, 0),
    center_of_dry_mass_position=0.42,
    grains_center_of_mass_position=0.42,
    grain_number=6,
    grain_separation=parameters.get("grain_separation")[0],
    grain_density=parameters.get("grain_density")[0],
    grain_outer_radius=parameters.get("grain_outer_radius")[0],
    grain_initial_inner_radius=parameters.get("grain_initial_inner_radius")[0],
    grain_initial_height=parameters.get("grain_initial_height")[0],
    nozzle_radius=parameters.get("nozzle_radius")[0],
    throat_radius=parameters.get("throat_radius")[0],
    interpolation_method="linear",
    nozzle_position=0,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

In [ ]:
keron.info()

### Rocket


Create the `Rocket` object


In [ ]:
valetudo = Rocket(
    radius=parameters.get("radius")[0],
    mass=parameters.get("rocket_mass")[0],
    inertia=(
        parameters.get("inertia_I")[0],
        parameters.get("inertia_I")[0],
        parameters.get("inertia_Z")[0],
    ),
    power_off_drag="../../data/rockets/valetudo/Cd_PowerOff_RASAero.csv",
    power_on_drag="../../data/rockets/valetudo/Cd_PowerOn_RASAero.csv",
    center_of_mass_without_motor=0,
)
valetudo.set_rail_buttons(0.224, -0.93, 30)
valetudo.add_motor(motor=keron, position=parameters.get("distance_rocket_nozzle")[0])

Adding aerodynamic surfaces


In [ ]:
nose_cone = valetudo.add_nose(
    length=parameters.get("nose_length")[0],
    kind="tangent",
    position=parameters.get("nose_distance_to_cm")[0],
)
fin_set = valetudo.add_trapezoidal_fins(
    3,
    span=parameters.get("fin_span")[0],
    root_chord=parameters.get("fin_root_chord")[0],
    tip_chord=parameters.get("fin_tip_chord")[0],
    position=parameters.get("fin_distance_to_cm")[0],
)

Adding Parachute


In [ ]:
drogue = valetudo.add_parachute(
    "Drogue",
    cd_s=parameters.get("cd_s_drogue")[0],
    trigger="apogee",
    sampling_rate=105,
    lag=parameters.get("lag_rec")[0],
    noise=(0, 8.3, 0.5),
)

Modify the Drag Coefficient curve


In [ ]:
valetudo.draw()

In [ ]:
valetudo.info()

### Flight


In [ ]:
test_flight = Flight(
    rocket=valetudo,
    environment=env,
    rail_length=parameters.get("rail_length")[0],
    inclination=parameters.get("inclination")[0],
    heading=parameters.get("heading")[0],
)

In [ ]:
test_flight.info()
test_flight.plots.trajectory_3d()

## Comparison with the real flight data


In [ ]:
# The flight recordings were lost after the launch. Only three useful information were recovered:
# - The apogee altitude (AGL): 860 m
# - East/West drift: 350 m
# - North/South drift: 25 m
# - Total drift: 350.9 m

In [ ]:
actual_data = {
    "apogee": 860,
    "east_west_drift": 350,
    "north_south_drift": 25,
    "total_drift": 350.9,
}

In [ ]:
simulated = {
    "apogee": test_flight.apogee - test_flight.env.elevation,
    "east_west_drift": test_flight.x(test_flight.t_final),
    "north_south_drift": test_flight.y(test_flight.t_final),
    "total_drift": test_flight.drift(test_flight.t_final),
}

In [ ]:
apogee_actual = actual_data.get("apogee")
apogee_simulated = simulated.get("apogee")
apogee_error = abs(apogee_actual - apogee_simulated)
apogee_percentage_error = apogee_error / apogee_actual * 100

print("Apogee (AGL): ")
print(f"Actual: {apogee_actual:.2f} m")
print(f"Simulated: {apogee_simulated:.2f} m")
print(f"Error: {apogee_error:.2f} m")
print(f"Percentage Error: {apogee_percentage_error:.2f}%")

In [ ]:
drift_actual = actual_data.get("total_drift")
drift_simulated = simulated.get("total_drift")
drift_error = abs(drift_actual - drift_simulated)
drift_percentage_error = drift_error / drift_actual * 100

print("Drift")
print(f"Actual: {drift_actual:.2f} m")
print(f"Simulated: {drift_simulated:.2f} m")
print(f"Error: {drift_error:.2f} m")
print(f"Percentage Error: {drift_percentage_error:.2f}%")
